In [2]:
def Phi1(T,AF,Q,Delta,Stock1,Stock2,Regime,Jump1,Jump2): 
    
    ###############################################################################
    #                           PARAMETER INPUT                                   #
    ###############################################################################
    #Stock1 = Stock(100,0.087**2,AF,0.06,0.14,3.46,0.006704,T,-0.82)
    #Stock2 = Stock(100,0.087**2,AF,0.03,0.14,3.46,0.002852,T,-0.82)
    #Regime = Regime2(Q);
    #S0, y0, AF, r, sigma, a, b, N, rho, mu,sigma_J,lambda_
    #Jump1 = Jump_Merton(100,0.087**2,AF,0.06,0.14,3.46,0.006704,T,-0.82,0.05,0.086,0.)
    #Jump2 = Jump_Merton(100,0.087**2,AF,0.03,0.14,3.46,0.002852,T,-0.82,0.06,0.086,0.3)
    #####################################################################################
    # ###############################Numerical Integration########################
    n = 10 # time step of integration
    X = np.linspace(T-Delta,T,n+1)
    phi1_1_2j = [];phi1_1_1j = [];phi1_1_0j = [];
    phi1_2_2j = [];phi1_2_1j = [];phi1_2_0j = []; 
    
    for i in range(len(X)):
        x1 = Jump1.L(-2j,X[i]); x2=Jump1.L(-1j,X[i]);x3=Jump1.L(0,X[i]);
        phi1_1_2j.append(x1); phi1_1_1j.append(x2); phi1_1_0j.append(x3);
        y1 = Jump2.L(-2j,X[i]); y2=Jump2.L(-1j,X[i]);y3=Jump2.L(0,X[i]);
        phi1_2_2j.append(y1); phi1_2_1j.append(y2);phi1_2_0j.append(y3); 
        
        
    phI1_1_2j = np.trapz(phi1_1_2j,dx=Delta/n);phI1_2_2j = np.trapz(phi1_2_2j,dx=Delta/n);
    phI1_1_1j = np.trapz(phi1_1_1j,dx=Delta/n);phI1_2_1j = np.trapz(phi1_2_1j,dx=Delta/n);
    phI1_1_0j = np.trapz(phi1_1_0j,dx=Delta/n);phI1_2_0j = np.trapz(phi1_2_0j,dx=Delta/n);
  
    #################################Diagonal Matrix#########################################
    phi1_Matrix_2j = np.diag(np.array([phI1_1_2j,phI1_2_2j]));
    phi1_Matrix_1j = np.diag(np.array([phI1_1_1j,phI1_2_1j]));
    phi1_Matrix_0j = np.diag(np.array([phI1_1_0j,phI1_2_0j]));
    
    #######################Phi1_characteristic function#####################################
    Phi1_2j = Regime.character(phi1_Matrix_2j,T-Delta,T);
    Phi1_1j = Regime.character(phi1_Matrix_1j,T-Delta,T);
    Phi1_0j = Regime.character(phi1_Matrix_0j,T-Delta,T);
    return Phi1_2j, Phi1_1j, Phi1_0j

In [3]:
def Phi2(T,AF,Q,Delta,Stock1,Stock2,Regime):
    
    ###############################################################################
    #                           PARAMETER INPUT                                   #
    ###############################################################################
    #
    #Stock1 = Stock(100,0.087**2,AF,0.06,0.14,3.46,0.006704,T,-0.82)# S0, y0, AF, r, sigma, a, b, N, rho
    #Stock2 = Stock(100,0.087**2,AF,0.03,0.14,3.46,0.002852,T,-0.82)
    #Regime = Regime2(Q);
    #
    ###############################################################################
    
    n = 10 # time step of integration
    X = np.linspace(0,T-Delta,n+1)
    phi2_1_2j = [];phi2_2_2j = [];
    
    for i in range(len(X)):
        H1 = Stock1.H(X[i]);H2 = Stock2.H(X[i]);
        x = Stock1.a*Stock1.b*H1;y = Stock2.a*Stock2.b*H2;
        phi2_1_2j.append(x);phi2_2_2j.append(y);
        #print(H1,X[i],T-Delta)
        
    phI2_1_2j = np.trapz(phi2_1_2j,dx=(T-Delta)/n);phI2_2_2j = np.trapz(phi2_2_2j,dx=(T-Delta)/n);
    phi2_Matrix = np.diag(np.array([phI2_1_2j,phI2_2_2j]))
    Phi2 = Regime.character(phi2_Matrix,0,T-Delta)
    return Phi2,Stock1.H(0)

In [4]:
def regime_VS(AF,Q):
    
    ###############################################################################
    #                           PARAMETER INPUT                                   #
    ###############################################################################
    T = 1; UK = 0; U = 0; 
    Regime = Regime2(Q);
    J = One_Path(Q,0,T)
    for k in range(0,AF*T):
        #Delta=1/AF
        #t_k=(k+1)*Delta
        #J = One_Path(Q,0,t_k)
        uk1 = path_one(AF,Q,k)     #Is there any improvement of efficiency we can make here ?
        uk2 = path_two(AF,Q,k)
        UK = UK + J[0]*uk1[0]+J[1]*uk2[0]
        #U = U + UK
        #print(uk1[0],uk2[0],UK)
        
    return UK   

In [5]:
def path_one(AF,Q,k):
    Regime = Regime2(Q);
    Delta = 1/AF
    t_k = (k+1)*Delta
    Stock1 = Stock(100,0.087**2,AF,0.03,0.14,3.46,0.002852,t_k,-0.82)# S0, y0, AF, r, sigma, a, b, T, rho
    Stock2 = Stock(100,0.087**2,AF,0.03,0.14,3.46,0.002852,t_k,-0.82)
    Jump1 = Jump_Merton(100,0.087**2,AF,0.03,0.14,3.46,0.002852,t_k,-0.82,0.05,0.086,0.1)
    Jump2 = Jump_Merton(100,0.087**2,AF,0.03,0.14,3.46,0.002852,t_k,-0.82,0.05,0.086,0.1)
    R = np.diag([np.exp(Stock1.r*Delta),np.exp(Stock2.r*Delta)])# matrix of interest rate
    Phi1_2j,Phi1_1j,Phi1_0j =  Phi1(t_k,AF,Q,Delta,Stock1,Stock2,Regime,Jump1,Jump2) 
    Phi2_,H1 = Phi2(t_k,AF,Q,Delta,Stock1,Stock2,Regime)
    if t_k == Delta:
        M = Stock1.M(-2j,0)
        uk = Phi1_2j[1]*np.exp(M*Stock1.y0)-2*Phi1_1j[1]+Phi1_0j[1]
        #Uk = np.matmul(R,uk)
        Uk = uk
    else:
        uk = np.multiply(Phi1_2j[1],Phi2_[1])*np.exp(H1*Stock1.y0)-2*Phi1_1j[1]+Phi1_0j[1]
    return uk

In [6]:
def path_two(AF,Q,k):
    Regime = Regime2(Q);
    Delta = 1/AF
    t_k = (k+1)*Delta
    Stock1 = Stock(100,0.087**2,AF,0.06,0.14,3.46,0.006704,t_k,-0.82)# S0, y0, AF, r, sigma, a, b, T, rho
    Stock2 = Stock(100,0.087**2,AF,0.06,0.14,3.46,0.006704,t_k,-0.82)
    Jump1 = Jump_Merton(100,0.087**2,AF,0.06,0.14,3.46,0.006704,t_k,-0.82,0.05,0.086,0.2)
    Jump2 = Jump_Merton(100,0.087**2,AF,0.06,0.14,3.46,0.006704,t_k,-0.82,0.05,0.086,0.2)
    R = np.diag([np.exp(Stock1.r*Delta),np.exp(Stock2.r*Delta)])# matrix of interest rate
    Phi1_2j,Phi1_1j,Phi1_0j = Phi1(t_k,AF,Q,Delta,Stock1,Stock2,Regime,Jump1,Jump2) 
    Phi2_,H1 = Phi2(t_k,AF,Q,Delta,Stock1,Stock2,Regime)
    if t_k == Delta:
        M = Stock1.M(-2j,0)
        uk = Phi1_2j[1]*np.exp(M*Stock1.y0)-2*Phi1_1j[1]+Phi1_0j[1]
        #Uk = np.matmul(R,uk)
        Uk = uk
    else:
        uk = np.multiply(Phi1_2j[1],Phi2_[1])*np.exp(H1*Stock1.y0)-2*Phi1_1j[1]+Phi1_0j[1]
    return uk

In [7]:
def One_Path(Q,j,T):
    J = np.array([0.00,0.00]) #initialize the waiting time
    t = T; # The maturity 
    I = j; # The initial state, choose 0 or 1
    while (t > 0):
        R1,R2 = np.random.uniform(0,1,2)
        tau = np.log(R1)/Q[I,I]
        pr = 0; m = 0;
        while (R2 > pr):
           # print(m,I)
            if m != I:
                pr = pr - Q[I,m]/Q[I,I];
                #print(pr)
            else:
                m = m+1;

        #print(tau,t) 
        if tau > t:
            J[I] = J[I]+t
            #print(J,I,t)
        else:
            J[I] = J[I]+tau
        t = t-tau; I = m;
    return J


In [8]:
# final main()
def semi_MC():
    Kvar = []
    AF = range(5,251,25)
    #AF = 10
    X = list(AF)
    Q = np.array([[-0.1,0.1],[0.4,-0.4]])
    # calculate discrete sols based AF
    for i in AF:
        Kvar_d = regime_VS(i,Q)
        #K_d = np.real(Kvar_d)
        Kvar.append(Kvar_d*10000)
    return  Kvar

In [9]:
from VS_class2 import Stock, Regime2, Jump_Merton, Jump_Kou
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy import linalg

Kvar = []
# generate 1000 sample paths
for m in range(10000):
    K = semi_MC()
    Kvar.append(K)
# evaluate the mean   
K = np.mean(np.array(Kvar),axis=0)
Outfile=open('Kvar_s.txt','a+')# write out the solution in txt file automatically
Outfile.write(str(K))
Outfile.close()    
print(K)

[55.89294645+0.j 28.08952419+0.j 26.20459365+0.j 25.5171422 +0.j
 25.16285018+0.j 24.93906874+0.j 24.80506733+0.j 24.69363963+0.j
 24.60994339+0.j 24.55518979+0.j]


In [27]:
# import pandas as ps
# import matplotlib.pyplot as plt
# yields = ps.read_csv('Book11.csv')
# yields2 = ps.read_csv('Book12.csv')
# plt.plot(yields['X'],yields['Kvar1'])
# plt.scatter(yields2['X'],yields2['Kvar1'])
# plt.show()

In [28]:
AF = range(5,251,25)
print(list(AF))

[5, 30, 55, 80, 105, 130, 155, 180, 205, 230]
